In [1]:
import torch
import torchaudio
import torch.nn.functional as F

import os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from IPython.display import Audio
import time

## 1. Creating the Dataset

### 2. HuBERT TEST

In [2]:
from transformers import HubertModel,AutoProcessor,AutoFeatureExtractor,Wav2Vec2Processor,HubertForCTC
import joblib
import torch.nn as nn
from hubertKM import SemanticTokenizer, visualizeEmbeddings

c:\Users\fabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Importing the pretrained models for HuBERT and KMeans

In [3]:
# A semantic tokenizer
# Input: (w2vCheckpointPath, kmeansCheckpointPath)
# Output: (semanticTokens, normalizedEmbeddings)
w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./hubertKM/hubert_base_ls960_L9_km500.bin")  

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

#### Computing semantic tokens

In [4]:
#semanticTokens, embeddings

#### Visualizing embeddings

In [5]:
# Create some plots using PCA and t-SNE

# visualizeEmbeddings(embeddings, semanticTokens)

#### Check with another implementation (OPTIONAL)

## 3. Creating the Semantic Transformer

In [6]:
from SoundStream import soundstream_16khz, audio_to_tokens, tokens_to_audio, encode_audio, decode_audio

In [7]:
soundStream = soundstream_16khz()

## TEST

In [8]:
from data import storeTokens,  TokensDataset

tokenPath = "out"
tokenFile = "out.csv"
audioPath = "data_cut"


In [9]:
#fileCount = storeTokens(audioPath, tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10)

In [10]:
#tokenDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 30, includeSemanticTokens = True, includeCoarseTokens = True, includeFineTokens = True) 
#semanticDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 30, includeSemanticTokens = True)
coarseDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 10, includeCoarseTokens = True)
#fineDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 30, includeFineTokens = True)

In [11]:
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from TransformerModel import Decoder
import pytorch_lightning as pl

In [12]:
#dataset = semanticDataset
train_dataset = coarseDataset

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

input_size = 0
for batch in train_loader:
    input, labels = batch
    print(input.requires_grad)
    break


False


In [13]:

#Change vocab_size and seq_len to initiate
model = Decoder(vocab_size=10000, seq_len=1503)


In [16]:
model.train(train_dataset)

100%|██████████| 55/55 [3:34:32<00:00, 234.05s/it]  


Epoch 1/10, Loss: 7.94355732310902
Checkpoint saved at ./checkpoints\checkpoint_epoch_1.pt


  9%|▉         | 5/55 [18:08<2:57:12, 212.65s/it]

In [16]:
#model = Decoder.load_from_checkpoint(checkpoint_path="checkpoints\checkpoint_epoch_1.pt")

checkpoint = torch.load("checkpoints\checkpoint_epoch_1.pt")
model = Decoder(vocab_size=10000, seq_len=1503)  # Initialize your model
model.load_state_dict(checkpoint['model_state_dict'])

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\fabri\AppData\Local\Temp\ipykernel_15548\3471269155.py:3: SyntaxWarning: invalid escape sequence '\c'
  checkpoint = torch.load("checkpoints\checkpoint_epoch_1.pt")


<All keys matched successfully>

In [19]:
#model.eval()
x_input = coarseDataset.__getitem__(0)

print(x_input)
with torch.no_grad():
	predictions = model(x_input[0])

(tensor([[ 516, 1284, 2228,  ...,  962, 1127, 2489]]), tensor([[1284, 2228,  605,  ..., 1127, 2489, -100]]))


In [22]:
pred = torch.argmax(predictions, dim=-1)
pred.shape

torch.Size([1, 1503])

In [23]:
fineDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 10, includeFineTokens = True)

In [44]:
y_input = fineDataset.__getitem__(0)[0]

y_input = y_input.squeeze(0)
pred_new = pred.squeeze(0)
#waveform = tokens_to_audio(pred_new, y_input, soundStream)
y_input.shape

torch.Size([2505])